In [ ]:
#panel de datos: balanceados: individuos son observados en el mismo periodo de tiempo
#no balanceados: el número de periodos para cada individuo es diferente
#Panel corto: N es pequeño (individuos) T es grande (periodos)
#panel largo: N es grande (individuos) T es pequeño (periodos)

In [ ]:
#variables explciativas: 
#1. varian en el tiempo y entre individuos: overall
#2. varían entre individuos pero constantes en el tiempo: between
#3. Constantes para todos los individuos pero Varían en el tiempo:  within

In [ ]:
#Modelos:
#1. agrupados (corte transversal, MCo, no considera las dos dimensiones, el corte transversal y el temporal)
#2. fijos (efectos  individuales): los efectos individuales están correlacionados con las variables explicativas
#3. aleatorios (efectos individuales): los efectos individuales no están correlacionados con las variables explicativas, se incluye en el termino de error.
#los modelos 2 y 3 reconocen la heterogeneidad entre los individuos a través del intercepto (alpha cambia para cada individuo)

In [ ]:
#estimadores: #estimadores: 
#MCO agrupado: incositente si hay efectos fijos, es decir si no hay variación entre individuos pero si en el tiempo
#between: promedio individual de cada variable, se tiene una observación para cada individuo, desaparece la variación temporal
#efectos fijos o within: a cada observación se le resta el promedio, se elimina el alpha.
#primeras diferencias: al valor actual le resta el valor anterior, desaparece alpha, no se pueden incluir time invariant regressor
#efectos aleatorios: quita una parte del promedio, se pueden incluir variables que son constantes en el tiempo.

In [1]:
!pip install linearmodels

     |████████████████████████████████| 1.5MB 5.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import linearmodels as lm

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [5]:
data = pd.read_csv("panel_wage.csv")

In [6]:
data.head()

,exp,wks,occ,ind,south,smsa,ms,fem,union,ed,...,id,t,tdum1,tdum2,tdum3,tdum4,tdum5,tdum6,tdum7,exp2
0,3,32,0,0,1,0,1,0,0,9,...,1,1,1,0,0,0,0,0,0,9
1,4,43,0,0,1,0,1,0,0,9,...,1,2,0,1,0,0,0,0,0,16
2,5,40,0,0,1,0,1,0,0,9,...,1,3,0,0,1,0,0,0,0,25
3,6,39,0,0,1,0,1,0,0,9,...,1,4,0,0,0,1,0,0,0,36
4,7,42,0,1,1,0,1,0,0,9,...,1,5,0,0,0,0,1,0,0,49


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4165 entries, 0 to 4164
Data columns (total 22 columns):
exp      4165 non-null int64
wks      4165 non-null int64
occ      4165 non-null int64
ind      4165 non-null int64
south    4165 non-null int64
smsa     4165 non-null int64
ms       4165 non-null int64
fem      4165 non-null int64
union    4165 non-null int64
ed       4165 non-null int64
blk      4165 non-null int64
lwage    4165 non-null float64
id       4165 non-null int64
t        4165 non-null int64
tdum1    4165 non-null int64
tdum2    4165 non-null int64
tdum3    4165 non-null int64
tdum4    4165 non-null int64
tdum5    4165 non-null int64
tdum6    4165 non-null int64
tdum7    4165 non-null int64
exp2     4165 non-null int64
dtypes: float64(1), int64(21)
memory usage: 715.9 KB


In [8]:
data.columns


Index(['exp', 'wks', 'occ', 'ind', 'south', 'smsa', 'ms', 'fem', 'union', 'ed',
       'blk', 'lwage', 'id', 't', 'tdum1', 'tdum2', 'tdum3', 'tdum4', 'tdum5',
       'tdum6', 'tdum7', 'exp2'],
      dtype='object')

In [13]:
data.index

RangeIndex(start=0, stop=4165, step=1)

In [14]:
#se convierte el indice a fechas para la dimensión temporal
#indexar la información de acuerdo al individuo al que pertenece y al periodo de tiempo
#id: individuo
#t: periodo
paneldata = data.set_index(["id", "t"])

In [17]:
paneldata.tail()

exp  wks  occ  ind  south  smsa  ms  fem  union  ed  blk    lwage  \
id  t                                                                      
595 3    3   50    0    0      0     1   0    1      0  12    0  5.95324   
    4    4   49    0    0      0     1   0    1      0  12    0  6.06379   
    5    5   50    0    0      0     1   0    1      0  12    0  6.21461   
    6    6   50    0    0      0     1   0    1      0  12    0  6.29157   
    7    7   50    0    0      0     1   0    1      0  12    0  6.37161   

       tdum1  tdum2  tdum3  tdum4  tdum5  tdum6  tdum7  exp2  
id  t                                                         
595 3      0      0      1      0      0      0      0     9  
    4      0      0      0      1      0      0      0    16  
    5      0      0      0      0      1      0      0    25  
    6      0      0      0      0      0      1      0    36  
    7      0      0      0      0      0      0      1    49

In [19]:
paneldata.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4165 entries, (1, 1) to (595, 7)
Data columns (total 20 columns):
exp      4165 non-null int64
wks      4165 non-null int64
occ      4165 non-null int64
ind      4165 non-null int64
south    4165 non-null int64
smsa     4165 non-null int64
ms       4165 non-null int64
fem      4165 non-null int64
union    4165 non-null int64
ed       4165 non-null int64
blk      4165 non-null int64
lwage    4165 non-null float64
tdum1    4165 non-null int64
tdum2    4165 non-null int64
tdum3    4165 non-null int64
tdum4    4165 non-null int64
tdum5    4165 non-null int64
tdum6    4165 non-null int64
tdum7    4165 non-null int64
exp2     4165 non-null int64
dtypes: float64(1), int64(19)
memory usage: 667.8 KB


In [20]:
paneldata.index

MultiIndex(levels=[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

In [21]:
y = paneldata["lwage"]
x = paneldata[["exp", "exp2", "wks", "ed"]]

In [22]:
#minimos cuadrados agrupados
mco_agrupados = lm.PooledOLS(y, sm.add_constant(x)) #objeto modelo
results_mco_a =  mco_agrupados.fit() #estimación de los parametros del objeto modelo
print(results_mco_a.summary) # summary de linear models es un atributo, en statsmodeles es un metodo

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.2836
Estimator:                  PooledOLS   R-squared (Between):              0.3163
No. Observations:                4165   R-squared (Within):               0.1957
Date:                Mon, May 11 2020   R-squared (Overall):              0.2836
Time:                        00:27:07   Log-likelihood                   -1994.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      411.62
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(4,4160)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             411.62
                            

In [23]:
#panel balaneado
results_mco_a.entity_info

mean        7.0
median      7.0
max         7.0
min         7.0
total     595.0
Name: Observations per entity, dtype: float64

In [24]:
#estimado between
modelo_between = lm.BetweenOLS(y, sm.add_constant(x))
results_between = modelo_between.fit()
print(results_between.summary)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.3264
Estimator:                 BetweenOLS   R-squared (Between):              0.3264
No. Observations:                 595   R-squared (Within):               0.1264
Date:                Mon, May 11 2020   R-squared (Overall):              0.2721
Time:                        00:35:16   Log-likelihood                   -172.39
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      71.477
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(4,590)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             71.477
                            

In [25]:
results_between.nobs #número de observaciones = al número de individuos. Promedia los periodos de tiempo.

595

In [26]:
results_mco_a.nobs

4165

In [50]:
#estimador efectos fijos, within, variable varia en tiempo pero no entre individuos
modelo_efectos_fijos = lm.PanelOLS(y, x, entity_effects = True) # permite capturar la heterogeidad entre los individuos,inluir los efectos individuales 
resultados_efectos_fijos = modelo_efectos_fijos.fit()
print(resultados_efectos_fijos.summary)
#el modelo no puede incluir variables que no presentan variación within a lo largo del tiempo

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          ed

Set drop_absorbed=True to automatically drop absorbed variables.


In [30]:
#ver si la variable ed presenta variación. Desviación estandar entre individuos
paneldata["ed"].std()
# la variable ed varia entre individuos pero para el mismo individuo es constante

2.787995034697743

In [34]:
#desviación estandar para cada individuo.
print(paneldata["ed"].groupby("id").std().max())  #valor maximo de las desviaciones estandar para cada individuo
print(paneldata["ed"].groupby("id").std().min())  #valor minimo de las desviaciones estandar para cada individuo
print(paneldata["ed"].groupby("id").std().mean())  #media de las desviaciones estandar para cada individuo

0.0
0.0
0.0


In [51]:
#excluir la constante ed que no presenta variación within
modelo_efectos_fijos = lm.PanelOLS(y, paneldata[["exp", "exp2", "wks"]], entity_effects = True) # permite capturar la heterogeidad entre los individuos,inluir los efectos individuales 
resultados_efectos_fijos = modelo_efectos_fijos.fit()
print(resultados_efectos_fijos.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.6566
Estimator:                   PanelOLS   R-squared (Between):              0.5037
No. Observations:                4165   R-squared (Within):               0.6566
Date:                Mon, May 11 2020   R-squared (Overall):              0.5039
Time:                        02:15:40   Log-likelihood                    2253.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2273.7
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(3,3567)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             2273.7
                            

In [37]:
#efectos aleatorios. No hay probelmas con variables constantes. Resta una parte del prmedio a cada observación.
modelo_efectos_aleatorios = lm.RandomEffects(y, sm.add_constant(x))
resultados_efectos_aleatorios = modelo_efectos_aleatorios.fit()
print(resultados_efectos_aleatorios.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.4200
Estimator:              RandomEffects   R-squared (Between):             -1.1061
No. Observations:                4165   R-squared (Within):               0.5487
Date:                Mon, May 11 2020   R-squared (Overall):             -0.6571
Time:                        01:49:32   Log-likelihood                    993.25
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      753.01
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(4,4160)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             753.01
                            

In [38]:
#descomposición de la varianza.
resultados_efectos_aleatorios.variance_decomposition. #El 81% de la varinza corresponde a los efectos aleatorios

Effects                   0.102091
Residual                  0.023172
Percent due to Effects    0.815012
Name: Variance Decomposition, dtype: float64

In [39]:
resultados_efectos_aleatorios.theta.head() #el landha o theta
#parte que utiliza del promedio para estimar los efectos aleatorios

,theta
id,
1,0.82278
2,0.82278
3,0.82278
4,0.82278
5,0.82278


In [41]:
#primeras diferencias. Importa el tipo de variables, no se pude incluir ed, porque es constante a través del tiempo para los individuos
modelo_primeras_diferencias = lm.FirstDifferenceOLS(y, paneldata[["exp", "exp2", "wks"]])
resultados_primeras_diferencias = modelo_primeras_diferencias.fit()
print(resultados_primeras_diferencias.summary)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  lwage   R-squared:                        0.2209
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.4957
No. Observations:                3570   R-squared (Within):               0.6559
Date:                Mon, May 11 2020   R-squared (Overall):              0.4959
Time:                        01:58:31   Log-likelihood                    1027.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      337.12
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(3,3567)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             337.12
                            

In [43]:
resultados_primeras_diferencias.nobs #595 indiiduos por 7 periodos de tiempo
# La primera difernecia es el valor actual menos el anterior. Se pierde la diferencia de la última observación(si es serie, para panel se pierde la primera observación de la tabla para cada individuo) para cada uno de los individuos: 595

3570

In [52]:
#comparar los 4 modelos en una sola tabla
from linearmodels.panel import compare

In [54]:
#claves del diccionario son los nombres de los modelos, con sus respectivas estancias de resultados
diccionario = {"MCO Agrupados":results_mco_a,
              "Between":results_between,
              "Efectos fijos Within":resultados_efectos_fijos,
              "Efectos aleatorios":resultados_efectos_aleatorios,
              "Primeras diferencias":resultados_primeras_diferencias}


In [59]:
print(compare(diccionario, precision = "std_errors")) #reportar errores estandar, por defecto imprime el estadistico t
print(compare(diccionario, precision = "pvalues"))

                                                   Model Comparison                                                  
                          MCO Agrupados        Between Efectos fijos Within Efectos aleatorios   Primeras diferencias
---------------------------------------------------------------------------------------------------------------------
Dep. Variable                     lwage          lwage                lwage              lwage                  lwage
Estimator                     PooledOLS     BetweenOLS             PanelOLS      RandomEffects     FirstDifferenceOLS
No. Observations                   4165            595                 4165               4165                   3570
Cov. Est.                    Unadjusted     Unadjusted           Unadjusted         Unadjusted             Unadjusted
R-squared                        0.2836         0.3264               0.6566             0.4200                 0.2209
R-Squared (Within)               0.1957         0.1264  